# 自动求导
假设我们想对函数y=2x⊤x关于列向量x求导

In [2]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于x的梯度之前，我们需要一个地方来存储梯度

In [4]:
x.requires_grad_(True)#等价于x = torch.arange(4.0,requires_grade=True)
x.grad #梯度是存在这里的，初始化是None

来计算y

In [7]:
y = 2 * torch.dot(x,x)
y #由于是隐式计算grad，是通过grad_fn=<MulBackward0>

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于x每个分量的梯度

In [8]:
y.backward()#求导
x.grad#来访问导数


tensor([ 0.,  4.,  8., 12.])

In [9]:
#检验一下导数计算是否正确
x.grad == 4 * x

tensor([True, True, True, True])

现在让我们计算x的另一个函数

In [10]:
# pytorch 会自动累积梯度，所以在计算另一个函数的梯度的时候，需要清除之前的梯度
x.grad.zero_()

y = x.sum()
y.backward()

x.grad

tensor([1., 1., 1., 1.])

深度学习中 ，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

In [11]:
x.grad.zero_()
y = x * x #y不是一个scalar，而是一个向量/矩阵的，按理说求梯度，得到的是微分矩阵，但是一般不这么做，后面解释
y.sum().backward()#对y求sum得到一个scalar，再算梯度
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外

In [12]:
x.grad.zero_()
y = x * x
u = y.detach() # u在计算图之外
z = u * x#u对x来说就是一个常数

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [13]:
#y在计算图里面，可以求得x的梯度
y.sum().backward()
x.grad == 2*x

tensor([ True, False, False, False])

即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度

In [14]:
def f(a):
    b = a * 2
    while b.norm() < 1000:#当norm小于1000时，一直x2
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)#torch[],size=0都代表一个scalar
d = f(a)
d.backward()

a.grad == d / a # 一次 函数


tensor(True)